In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr
import matplotlib.pyplot as plt


In [84]:
biden_v_trump24 = pd.read_csv('../election_resources/biden_v_trump24.csv')
biden_v_trump24.head()

,question_id,end_date,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,178646,2023-08-08,YouGov,1500.0,a,Joe Biden,44.0,53.0,20.0,24.0,13.0,40.0,0
1,178647,2023-08-08,YouGov,1178.0,rv,Joe Biden,45.0,53.0,23.0,22.0,11.0,42.0,0
2,178439,2023-08-03,Ipsos,1076.0,a,Joe Biden,33.0,52.0,0.0,0.0,0.0,0.0,0
3,178683,2023-08-03,Cygnal Political,2000.0,lv,Joe Biden,45.9,51.3,25.3,20.7,12.4,38.9,0
4,178365,2023-08-01,YouGov,1500.0,a,Joe Biden,42.0,55.0,18.0,24.0,12.0,43.0,0


In [85]:
biden_v_trump24.drop(columns=['question_id', 'end_date'], inplace=True)
biden_v_trump24.fillna(0, inplace=True)
biden_v_trump24.head()

,pollster,sample_size,population,politician,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results
0,YouGov,1500.0,a,Joe Biden,44.0,53.0,20.0,24.0,13.0,40.0,0
1,YouGov,1178.0,rv,Joe Biden,45.0,53.0,23.0,22.0,11.0,42.0,0
2,Ipsos,1076.0,a,Joe Biden,33.0,52.0,0.0,0.0,0.0,0.0,0
3,Cygnal Political,2000.0,lv,Joe Biden,45.9,51.3,25.3,20.7,12.4,38.9,0
4,YouGov,1500.0,a,Joe Biden,42.0,55.0,18.0,24.0,12.0,43.0,0


In [6]:
politicians_to_replace = list(biden_v_trump24['politician'].value_counts()[biden_v_trump24['politician'].value_counts() < 5].index)
for i in politicians_to_replace:
    biden_v_trump24['politician'] = biden_v_trump24['politician'].replace(i, 'Other')

In [86]:
pollster_to_replace = list(biden_v_trump24['pollster'].value_counts()[biden_v_trump24['pollster'].value_counts() < 33].index)
for i in pollster_to_replace:
    biden_v_trump24['pollster'] = biden_v_trump24['pollster'].replace(i, 'Other')

In [87]:
biden_v_trump24 = pd.get_dummies(biden_v_trump24, dtype=float)
biden_v_trump24.head()

,sample_size,favorable,unfavorable,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,results,pollster_Echelon Insights,pollster_Global Strategy Group/GBAO/Navigator Research,...,pollster_Morning Consult,pollster_Other,pollster_Quinnipiac,pollster_YouGov,population_a,population_lv,population_rv,population_v,politician_Donald Trump,politician_Joe Biden
0,1500.0,44.0,53.0,20.0,24.0,13.0,40.0,0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1178.0,45.0,53.0,23.0,22.0,11.0,42.0,0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1076.0,33.0,52.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,2000.0,45.9,51.3,25.3,20.7,12.4,38.9,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1500.0,42.0,55.0,18.0,24.0,12.0,43.0,0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [88]:
biden_v_trump24.shape

(1662, 23)

In [89]:
nn_model = tf.keras.models.load_model('trained_model_president.h5')
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 93)                2232      
                                                                 
 dense_12 (Dense)            (None, 77)                7238      
                                                                 
 dense_13 (Dense)            (None, 79)                6162      
                                                                 
 dense_14 (Dense)            (None, 53)                4240      
                                                                 
 dense_15 (Dense)            (None, 97)                5238      
                                                                 
 dense_16 (Dense)            (None, 15)                1470      
                                                                 
 dense_17 (Dense)            (None, 33)               

In [90]:
scaler = StandardScaler()
X_scaler = scaler.fit(biden_v_trump24)
X_scaled = X_scaler.transform(biden_v_trump24)


In [91]:
predictions = nn_model.predict(X_scaled)

52/52 [==============================] - 2s 2ms/step


In [92]:
predictions

array([[9.9733186e-01],
       [9.9794012e-01],
       [9.9822468e-01],
       ...,
       [4.6642763e-05],
       [4.2523967e-05],
       [4.4681845e-05]], dtype=float32)

In [93]:
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [94]:
predictions_df.to_csv('predictions.csv', index=False)

In [95]:
predictions_df.head()

,predictions
0,0.997332
1,0.997940
2,0.998225
3,0.989938
4,0.995206
